In [1]:
import warnings
warnings.filterwarnings("ignore")

import logging
import pandas as pd
import os
import gzip

from datetime import datetime
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams

# importo los .py
import sys
sys.path.append("auxiliares")
import metricas

In [2]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/diego_tools/labo3/")
os.chdir("/home/dcastro_contacto/buckets/b1/")
####################################################

In [3]:
MES_TRAIN_TOPE = 201912
PERIODO_ESTACIONAL = 201902
PERIODO_MENOS_2 = 201912

In [ ]:
star_products = [20001, 20002, 20003, 20004, 20005, 20006, 20007, 20009, 20011, 20032]
star_customers = [10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009, 10011, 10012, 10013]

In [4]:
arch_prod_ids_prediccion = "datasets/productos_a_predecir_201912.csv"
arch_sellout_completo = "datasets/emp3_sellout_base.csv"

carpeta_base_exp = "exp/basica_naif/"

In [5]:
df_sellout = pd.read_csv(arch_sellout_completo)

In [6]:
df_sellout.head()

,product_id,periodo,customer_id,cero_ventas,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,periodo_fecha,mes,...,cat3,brand,sku_size,producto_estrella,temp_media,temp_max_media,temp_min_media,IPC,promedio_mens_dolar_venta,catastrofe
0,20001,201701,10001,0,0.0,11.0,99.43861,99.43861,2017-01-01,1,...,Liquido,ARIEL,3000.0,1.0,25.65,30.4,20.9,1.6,15.91,False
1,20001,201702,10001,0,0.0,23.0,198.84365,198.84365,2017-02-01,2,...,Liquido,ARIEL,3000.0,1.0,25.35,30.1,20.6,2.1,15.59,False
2,20001,201703,10001,0,0.0,33.0,92.46537,92.46537,2017-03-01,3,...,Liquido,ARIEL,3000.0,1.0,22.45,26.9,18.0,2.4,15.52,False
3,20001,201704,10001,0,0.0,8.0,13.29728,13.29728,2017-04-01,4,...,Liquido,ARIEL,3000.0,1.0,18.90,23.5,14.3,2.7,15.36,False
4,20001,201705,10001,0,0.0,15.0,101.20711,101.00563,2017-05-01,5,...,Liquido,ARIEL,3000.0,1.0,15.90,19.5,12.3,1.4,15.72,False


In [7]:
df_sellout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18954127 entries, 0 to 18954126
Data columns (total 26 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   product_id                 int64  
 1   periodo                    int64  
 2   customer_id                int64  
 3   cero_ventas                int64  
 4   plan_precios_cuidados      float64
 5   cust_request_qty           float64
 6   cust_request_tn            float64
 7   tn                         float64
 8   periodo_fecha              object 
 9   mes                        int64  
 10  meses_historia_prod        int64  
 11  meses_historia_cliente     int64  
 12  cliente_estrella           int64  
 13  prod_cust                  object 
 14  cat1                       object 
 15  cat2                       object 
 16  cat3                       object 
 17  brand                      object 
 18  sku_size                   float64
 19  producto_estrella          float64
 20  

In [8]:
# Selecciono las columnas iniciales
df_sellout = df_sellout[["periodo","product_id","customer_id","tn"]]

print(len(df_sellout.product_id.unique()))
print(len(df_sellout.customer_id.unique()))

1233
597


## Train

In [9]:
df_train = df_sellout[df_sellout.periodo <= MES_TRAIN_TOPE]

In [11]:
print(len(df_train.product_id.unique()))
print(len(df_train.customer_id.unique()))
df_train.head()

1233
597


,periodo,product_id,customer_id,tn
0,201701,20001,10001,99.43861
1,201702,20001,10001,198.84365
2,201703,20001,10001,92.46537
3,201704,20001,10001,13.29728
4,201705,20001,10001,101.00563


In [12]:
df_prods_prediccion=pd.read_csv(arch_prod_ids_prediccion)
df_prods_prediccion.head()

,product_id
0,20001
1,20002
2,20003
3,20004
4,20005


In [13]:
len(df_prods_prediccion)

780

In [14]:
# Como puede haber productos que no estén en algunos períodos (que se usan en modelos naif), calculo promedio para completar
df_completar = df_train[["product_id","customer_id","tn"]].groupby(["product_id","customer_id"]).mean().reset_index()

print(len(df_completar.product_id.unique()))
print(len(df_completar.customer_id.unique()))

df_completar.head()

1233
597


,product_id,customer_id,tn
0,20001,10001,180.409285
1,20001,10002,39.873140
2,20001,10003,114.305746
3,20001,10004,218.406489
4,20001,10005,18.387957


## Predicción Estacional

In [15]:
df_pred_estac = df_train[df_train.periodo==PERIODO_ESTACIONAL]
df_pred_estac = df_pred_estac[["product_id","customer_id","tn"]]
df_pred_estac.head()

,product_id,customer_id,tn
25,20001,10001,309.90610
60,20001,10002,6.48602
95,20001,10003,81.21522
130,20001,10004,155.26111
165,20001,10005,4.55925


In [16]:
# Left join con prediccion promedio (para completar)
df_res_estac = pd.merge(df_completar,df_pred_estac,on=["product_id","customer_id"],how="left",suffixes=["_prom","_estac"])

# Calculo toneladas predichas usando los valores anteriores
df_res_estac["tn_pred"]=np.where(df_res_estac.tn_estac.isnull(),df_res_estac.tn_prom,df_res_estac.tn_estac)

df_res_estac = df_res_estac[["product_id","customer_id","tn_pred"]]
df_res_estac.head()

,product_id,customer_id,tn_pred
0,20001,10001,309.90610
1,20001,10002,6.48602
2,20001,10003,81.21522
3,20001,10004,155.26111
4,20001,10005,4.55925


In [17]:
df_res_estac.isna().sum()

product_id     0
customer_id    0
tn_pred        0
dtype: int64

In [18]:
df_res_estac = df_res_estac[df_res_estac.product_id.isin(df_prods_prediccion.product_id)]

In [19]:
print(len(df_res_estac))
df_res_estac = df_res_estac.rename(columns={"tn_pred":"tn"})
df_res_estac.head()

465660


,product_id,customer_id,tn
0,20001,10001,309.90610
1,20001,10002,6.48602
2,20001,10003,81.21522
3,20001,10004,155.26111
4,20001,10005,4.55925


In [22]:
df_pred_comp1 = df_res_estac.groupby("product_id")["tn"].sum().reset_index()
df_pred_comp1.to_csv(carpeta_base_exp + "naif_estac_avanzada_prediccion_comp1.csv",index=False)
df_pred_comp1

,product_id,tn
0,20001,1259.529979
1,20002,1044.028912
2,20003,760.513612
3,20004,442.971916
4,20005,411.500410
...,...,...
775,21263,0.059471
776,21265,0.097829
777,21266,0.103641
778,21267,0.098812


In [23]:
df_pred_comp2 = df_res_estac
df_pred_comp2["product_customer_id"]="P"+df_pred_comp2["product_id"].astype(str) + '-C' + df_pred_comp2["customer_id"].astype(str)
df_pred_comp2 = df_pred_comp2[(df_pred_comp2.product_id.isin(star_products)) & (df_pred_comp2.customer_id.isin(star_customers))]
df_pred_comp2 = df_pred_comp2.drop(columns=["product_id","customer_id"])
df_pred_comp2.to_csv(carpeta_base_exp + "naif_estac_avanzada_prediccion_comp2.csv",index=False)
print(df_pred_comp2.head())
print(len(df_pred_comp2))

          tn product_customer_id
0  309.90610       P20001-C10001
1    6.48602       P20001-C10002
2   81.21522       P20001-C10003
3  155.26111       P20001-C10004
4    4.55925       P20001-C10005
120


## Predicción Menos 2

In [24]:
# Naif -2
df_pred_menos2 = df_train[df_train.periodo==PERIODO_MENOS_2]
df_pred_menos2 = df_pred_menos2[["product_id","customer_id","tn"]]
df_pred_menos2.head()

,product_id,customer_id,tn
34,20001,10001,180.21938
69,20001,10002,113.33165
104,20001,10003,102.27517
139,20001,10004,34.64810
174,20001,10005,19.60368


In [25]:
# Left join con prediccion promedio (para completar)
df_res_menos2 = pd.merge(df_completar,df_pred_menos2,on=["product_id","customer_id"],how="left",suffixes=["_prom","_menos2"])

# Calculo toneladas predichas usando los valores anteriores
df_res_menos2["tn_pred"]=np.where(df_res_menos2.tn_menos2.isnull(),df_res_menos2.tn_prom,df_res_menos2.tn_menos2)

df_res_menos2 = df_res_menos2[["product_id","customer_id","tn_pred"]]
df_res_menos2.head()

,product_id,customer_id,tn_pred
0,20001,10001,180.21938
1,20001,10002,113.33165
2,20001,10003,102.27517
3,20001,10004,34.64810
4,20001,10005,19.60368


In [26]:
df_res_menos2 = df_res_menos2[df_res_menos2.product_id.isin(df_prods_prediccion.product_id)]

In [27]:
print(len(df_res_menos2))
df_res_menos2 = df_res_menos2.rename(columns={"tn_pred":"tn"})
df_res_menos2.head()

465660


,product_id,customer_id,tn
0,20001,10001,180.21938
1,20001,10002,113.33165
2,20001,10003,102.27517
3,20001,10004,34.64810
4,20001,10005,19.60368


In [29]:
df_pred_comp1 = df_res_menos2.groupby("product_id")["tn"].sum().reset_index()
df_pred_comp1.to_csv(carpeta_base_exp + "naif_menos2_avanzada_prediccion_comp1.csv",index=False)
df_pred_comp1

,product_id,tn
0,20001,1504.68856
1,20002,1087.30855
2,20003,892.50129
3,20004,637.90002
4,20005,593.24443
...,...,...
775,21263,0.01270
776,21265,0.05007
777,21266,0.05121
778,21267,0.01569


In [28]:
df_pred_comp2 = df_res_menos2
df_pred_comp2["product_customer_id"]="P"+df_pred_comp2["product_id"].astype(str) + '-C' + df_pred_comp2["customer_id"].astype(str)
df_pred_comp2 = df_pred_comp2[(df_pred_comp2.product_id.isin(star_products)) & (df_pred_comp2.customer_id.isin(star_customers))]
df_pred_comp2 = df_pred_comp2.drop(columns=["product_id","customer_id"])
df_pred_comp2.to_csv(carpeta_base_exp + "naif_menos2_avanzada_prediccion_comp2.csv",index=False)
print(df_pred_comp2.head())
print(len(df_pred_comp2))

          tn product_customer_id
0  180.21938       P20001-C10001
1  113.33165       P20001-C10002
2  102.27517       P20001-C10003
3   34.64810       P20001-C10004
4   19.60368       P20001-C10005
120
